In [1]:
import cirq
import numpy as np
import matplotlib.pyplot as plt 

from cirq import Circuit
from cirq.devices import GridQubit
from cirq.google import XmonSimulator
from cirq import InsertStrategy
from cirq.ops import CNOT, H, S

# Messy Compression, Large Circuit Depth for 2 and 3 Qubits

## Purpose
I'm putting some code here to try and create 2 and 3 Qubit circuits to simulate 4 and 8 Qubit spin-chains. These will get messy since I won't be converting the adiabatic steps into compact circuits, rather, I am simply going to make circuits with the full adiabatic steps and without the $W(J)$ reductions. Based on (Hebenstreit et al.)

In [2]:
qubits = cirq.LineQubit.range(3)
qubit0, qubit1, qubit2 = qubits

qubits2 = cirq.LineQubit.range(5)
q0, q1, q2, q3, q4 = qubits2
#circuit1 will be the two qubit circuit for quality purposes and circuit2 will be the 3 qubit circuit.
circuit1 = cirq.Circuit()
circuit2 = cirq.Circuit()

## Step 1
Prepare input state $\rho_{i n} = \frac{1}{2^{m-1}} I^{\otimes m-1} \otimes |+_{y} \rangle \langle+_{y} |$ for both circuits.

In [3]:
circuit1.append([cirq.H(qubit0)])
circuit1.append([cirq.S(qubit0)**-1])
circuit1.append([cirq.H(qubit2)], strategy = InsertStrategy.EARLIEST)
circuit1.append([cirq.CNOT(control=qubit2, target=qubit1)], strategy = InsertStrategy.EARLIEST)

circuit1

0: ───H───S^-1───

1: ───────X──────
          │
2: ───H───@──────

In [4]:
circuit2.append([cirq.H(q0), cirq.H(q1), cirq.H(q3), cirq.H(q4)])
circuit2.append([cirq.CNOT(control = q2, target = q0), cirq.S(q3)])
circuit2.append([cirq.CNOT(control = q2, target = q1)])
circuit2.append([H(q1), H(q2)])
circuit2.append([cirq.CNOT(control = q2, target = q1)])
circuit2.append([CNOT(control = q2, target = q4)])

circuit2

0: ───H───X───────────────────
          │
1: ───H───┼───X───H───X───────
          │   │       │
2: ───────@───@───H───@───@───
                          │
3: ───H───S───────────────┼───
                          │
4: ───H───────────────────X───

For time's sake right now, I'm only going to do the 3 qubit guy...

In [ ]:
#evolution parameters
L = 2400
dt = 0.1
def phase(J, L):
    